In [86]:
# Import libraries:
import numpy as np
import pandas as pd
import requests

Read the data from  Helsingin Sanomat (https://github.com/HS-Datadesk/koronavirus-avoindata)

In [87]:
data = requests.get("https://w3qa5ydb4l.execute-api.eu-west-1.amazonaws.com/prod/finnishCoronaData")
data_json = data.json()
#Take only the data for confirmed cases:
df = pd.DataFrame.from_dict(data_json["confirmed"])
print(df.head())
print(df.describe())

                       date healthCareDistrict id infectionSource  \
0  2020-01-29T11:00:00.000Z              Lappi  1         unknown   
1  2020-02-26T11:00:00.000Z                HUS  2         unknown   
2  2020-03-01T11:00:00.000Z                HUS  3               2   
3  2020-02-28T11:00:00.000Z                HUS  4         unknown   
4  2020-03-01T11:00:00.000Z                HUS  5               4   

  infectionSourceCountry  
0                    CHN  
1                    ITA  
2                    FIN  
3                    ITA  
4                    FIN  
                            date healthCareDistrict   id infectionSource  \
count                        223                223  223             223   
unique                        39                 16  223              11   
top     2020-03-14T09:20:00.000Z                HUS   95         unknown   
freq                          47                117    1             207   

       infectionSourceCountry  
count     

Now, we'll take only the dates and id's of the confirmed cases of COVID-19.
There are 223 confirmed cases with all having their own timestamps.

In [90]:
df_ts = df[['date','id']]
print(df_ts.head())
print(df_ts.dtypes)

                       date id
0  2020-01-29T11:00:00.000Z  1
1  2020-02-26T11:00:00.000Z  2
2  2020-03-01T11:00:00.000Z  3
3  2020-02-28T11:00:00.000Z  4
4  2020-03-01T11:00:00.000Z  5
date    object
id      object
dtype: object


Set the dataframe index to date.
Group the data by day to get a nice timeseries of the development of the virus in Finland.

In [91]:
new_dates = pd.Series([x[:10] for x in df_ts['date'].values],name='new_dates')
new_dates = new_dates.str.split('-',expand=True)
new_dates = new_dates.rename(columns = {0:'year',1:'month',2:'day'})
# print(new_dates)
df_ts = pd.concat([df_ts,new_dates],axis=1)
df_ts = df_ts.drop(columns = 'date')
df_ts["date"] = pd.to_datetime(df_ts[['year','month','day']])
covid_ts = df_ts.set_index('date')['id']
print(covid_ts.head())

date
2020-01-29    1
2020-02-26    2
2020-03-01    3
2020-02-28    4
2020-03-01    5
Name: id, dtype: object
